# Monthly Momentun Scanner Executed Weekly

In [1]:
import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
from stock_config import momentum_stock_list, sample_stock_list

## Input the evaluation date below (Every Friday date), stock universe and output filename

In [2]:
eval_date = "2020-09-28"

In [3]:
stock_list = sample_stock_list

In [4]:
print(f"Total stocks in the list {len(stock_list)}")

Total stocks in the list 3


In [5]:
filename = r'data/output/momentum_monthly_output_sample.xlsx'

#### Function to check valid trading day. If open price is not returned then next date is checked until valid date is found.
##### End date is valid date + 1. Used to get LTP for desired date using yfinance.

In [6]:
def get_valid_date(start_date, end_date):
    valid = False
    while not valid:
        valid_check_data = yf.download('RELIANCE.NS', start = start_date, end=end_date)
        try:
            open_price = valid_check_data.iloc[0]['Open']
            valid = True
        except:
            start_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            end_date = (pd.to_datetime(end_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
            valid = False
    return start_date, end_date

#### Get valid start and end dates for this week

In [7]:
start_date=eval_date
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
start_date, end_date = get_valid_date(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [8]:
start_date

'2020-09-28'

In [9]:
end_date

'2020-09-29'

### Fetch this week data

#### [Below data fetch will throw error if there is no trading day or if the scrip doesnt have ltp (not listed) between start and end dates]

In [10]:
data = yf.download(stock_list, start = start_date, end=end_date)

[*********************100%***********************]  3 of 3 completed


In [11]:
this_month_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [12]:
# this_month_close

#### Calculate dates for last week

In [13]:
start_date=(pd.to_datetime(eval_date) - pd.DateOffset(months=1)).strftime('%Y-%m-%d')
end_date = (pd.to_datetime(start_date) + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

#### Get valid start and end dates for last year

In [14]:
start_date, end_date = get_valid_date(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [15]:
start_date

'2020-08-28'

In [16]:
end_date

'2020-08-29'

### Fetch last year data

#### [Below data fetch will throw error if there is no trading day between start and end dates]

In [17]:
data = yf.download(stock_list, start = start_date, end=end_date)

[*********************100%***********************]  3 of 3 completed


In [18]:
last_month_close = pd.DataFrame(data.iloc[0]['Adj Close'])

In [19]:
# last_month_close

In [20]:
final_df = last_month_close.join(this_month_close)

In [21]:
final_df

,2020-08-28,2020-09-28
ADANIPORTS.NS,360.049988,339.549988
ASIANPAINT.NS,1955.115356,1942.634399
AXISBANK.NS,509.200012,439.250000


In [22]:
final_df.rename(columns =lambda t: t.strftime('%Y-%m-%d'), inplace=True)

## Calculate Momentum

In [23]:
last_month_date = final_df.columns[0]
this_month_date = final_df.columns[1]
final_df['Change'] = final_df[this_month_date] - final_df[last_month_date]

In [24]:
final_df['% Change'] = (final_df['Change'] / final_df[last_month_date])*100

## Final Output

In [25]:
final_df.sort_values(by=['% Change'], ascending=False, inplace=True)

In [26]:
final_df.rename_axis('MOMENTUM_ATH', inplace=True)
final_df.rename_axis('', axis='columns', inplace=True)

In [27]:
final_df.round(2).head(50)

,2020-08-28,2020-09-28,Change,% Change
MOMENTUM_ATH,,,,
ASIANPAINT.NS,1955.12,1942.63,-12.48,-0.64
ADANIPORTS.NS,360.05,339.55,-20.50,-5.69
AXISBANK.NS,509.20,439.25,-69.95,-13.74


### Create an empty excel file as below filename before running below code

In [28]:
book = load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book

In [29]:
final_df.round(2).to_excel(writer, sheet_name=eval_date)
writer.save()
writer.close()

In [30]:
f"Done. Output file path and name --> {filename}"

'Done. Output file path and name --> data/output/momentum_monthly_output_sample.xlsx'